In [5]:
# Conexión a Redshift

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import psycopg2
 
# conn var
host = '172.17.139.149'
port = 5439 
database = 'datasets'
user = 'gurenab'
password = 'Redshift2023.'
 
#create conn
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
        )
    print("Conexión exitosa a Redshift")
except mysql.connector.Error as err:
    print(f"Error al conectar a Redshift: {err}")

#Consulta SQL
query = "SELECT * FROM datasets.aioperations.portfoliobystate_dst;"
query = pd.read_sql_query(
                            """
                            SELECT
                            *
                            FROM datasets.aioperations.portfoliobystate_dst
                            """,
                            conn,
                        )
 
 
df = pd.DataFrame(query)
 
# Cerrar la conexión
conn.close()

Conexión exitosa a Redshift


C:\Users\gurenab\AppData\Local\Temp\ipykernel_78508\1390778327.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql_query(


In [48]:
import pandas as pd
import numpy as np

# Cargar los datos
CARTERA = pd.read_parquet("C:/Users/gurenab/Documents/Distribuciones/CARTERA.parquet")
CARTERA.info()

# Filtros
CARTERA = CARTERA[CARTERA['cli_original_code'].notna()]
CARTERA = CARTERA[CARTERA['countryname'] == "COSTA RICA"]
CARTERA = CARTERA[~CARTERA['fullname'].isin(["VEHICULOS VENDIDOS", "CUENTAS CANCELADAS CR", "VEHICULOS CAPTURADOS",
                                             "CUENTAS FALLECIDOS CR", "CORPORATIVO INACTIVO", "CUENTAS EXCLUIDAS CR",
                                             "CUENTAS EXCLUIDAS GT", "CUENTAS PRESCRITAS LEGALMENTE"])]

# Formato para cálculo disimilitudes
CARTERA['secondperiod_crdsrv'] = pd.to_datetime(CARTERA['secondperiod_crdsrv'], errors='coerce')
CARTERA['trabaja'] = np.where(CARTERA['secondperiod_crdsrv'] == CARTERA['secondperiod_crdsrv'].max(), "SI", "NO")
CARTERA['trabaja'] = CARTERA['trabaja'].fillna("NO")
CARTERA['trabaja'] = CARTERA['trabaja'].astype('category')

# Crear la columna 'expediente_valido'
valid_exp = ["90-", "91-", "92-", "93-", "94-", "95-", "96-", "97-", "98-", "99-", "00-", "01-", "02-", "03-", 
             "04-", "05-", "06-", "07-", "08-", "09-", "10-", "11-", "12-", "13-", "14-", "15-", "16-", "17-", 
             "18-", "19-", "20-", "21-", "22-", "23-", "24-", "25-", "26-", "27-", "28-", "29-"]

CARTERA['expediente_valido'] = np.where(CARTERA['courtarchive'].str.startswith(tuple(valid_exp)), "SI", "NO")
CARTERA['expediente_valido'] = CARTERA['expediente_valido'].astype('category')

# Crear la columna 'rango_salario'
CARTERA['rango_salario'] = pd.cut(CARTERA['lastsalary'], 
                                  bins=[-np.inf, 0, 300000, 450000, 600000, 1000000, np.inf], 
                                  labels=["NA_o_Negativo", "0_a_299_999", "300_000_a_449_999", 
                                          "450_000_a_599_999", "600_000_a_999_999", "1_000_000_y_mas"],
                                  ordered=True)

# Crear la columna 'rango_deuda'
CARTERA['rango_deuda'] = pd.cut(CARTERA['balance'], 
                                bins=[-np.inf, 0, 250000, 500000, 750000, 1000000, 1500000, 3000000, np.inf], 
                                labels=["NA o Negativo", "0_a_249_999", "250_000_a_499_999", 
                                        "500_000_a_749_999", "750_000_a_999_999", "1_000_000_a_1_499_999", 
                                        "1_500_000_a_2_999_999", "3_000_000_y_mas"],
                                ordered=True)

# Crear la columna 'cantidad_vehiculos'
CARTERA['cantidad_vehiculos'] = pd.cut(CARTERA['vehicules_crdsrv'].fillna(0), 
                                       bins=[-np.inf, 0, 1, 2, np.inf], 
                                       labels=["0", "1", "2", "3_o_mas"],
                                       ordered=True)

# Crear la columna 'cantidad_propiedades'
CARTERA['cantidad_propiedades'] = pd.cut(CARTERA['ownerships_crdsrv'].fillna(0), 
                                         bins=[-np.inf, 0, 1, 2, np.inf], 
                                         labels=["0", "1", "2", "3_o_mas"],
                                         ordered=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339065 entries, 0 to 4339064
Data columns (total 29 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   cli_identification         object        
 1   cli_original_code          object        
 2   fullname                   object        
 3   accounttype                object        
 4   countryname                object        
 5   institutiondescription     object        
 6   alchemydescription         object        
 7   operationnumber            object        
 8   include_date               datetime64[ns]
 9   statusdescription          object        
 10  currencyname               object        
 11  balance                    float64       
 12  originalamount             float64       
 13  interest                   float64       
 14  pendingcharges             float64       
 15  netamount                  float64       
 16  lastpayment_date           datetime6

In [ ]:
CARTERA

In [58]:
# Selección de variables para entrenar el modelo
base_dbscan = CARTERA[CARTERA['statusdescription'] == "Activa"]

# Selección de columnas
base_dbscan = base_dbscan[['operationnumber', 'trabaja', 'expediente_valido', 
                           'rango_salario', 'rango_deuda', 
                           'cantidad_vehiculos', 'cantidad_propiedades']]

In [50]:
base_dbscan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19814 entries, 11967 to 4309032
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   operationnumber       19814 non-null  object  
 1   trabaja               19814 non-null  category
 2   expediente_valido     19814 non-null  category
 3   rango_salario         18632 non-null  category
 4   rango_deuda           19814 non-null  category
 5   cantidad_vehiculos    19814 non-null  category
 6   cantidad_propiedades  19814 non-null  category
dtypes: category(6), object(1)
memory usage: 426.9+ KB


In [ ]:
base_dbscan

In [51]:
from sklearn.metrics import silhouette_score

# Definir rangos de eps y min_samples para buscar
eps_values = np.arange(0.1, 0.5, 0.05)
min_samples_values = range(5, 20, 2)

mejor_eps = None
mejor_min_samples = None
mejor_silueta = -1

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric="precomputed")
        agrupacion = dbscan.fit(gower_dist)
        labels = agrupacion.labels_
        
        # Excluir el ruido (label -1) al calcular la silueta
        if len(set(labels)) > 1:
            silueta = silhouette_score(gower_dist, labels, metric="precomputed")
            if silueta > mejor_silueta:
                mejor_eps = eps
                mejor_min_samples = min_samples
                mejor_silueta = silueta

print(f"Mejor eps: {mejor_eps}")
print(f"Mejor min_samples: {mejor_min_samples}")
print(f"Mejor silueta: {mejor_silueta}")

Mejor eps: 0.1
Mejor min_samples: 5
Mejor silueta: 0.6494431495666504


In [64]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances, silhouette_samples
from sklearn.cluster import DBSCAN
from gower import gower_matrix
from datetime import datetime

# Convertir las columnas categóricas a strings
base_dbscan = base_dbscan.copy()
for col in base_dbscan.select_dtypes(['category']).columns:
    base_dbscan[col] = base_dbscan[col].astype(str)

# Cálculo de disimilitudes utilizando la métrica de Gower
inicio = datetime.now()
gower_dist = gower_matrix(base_dbscan.iloc[:, 1:])
fin = datetime.now()
tiempo_distancias_activas = fin - inicio
print("Tiempo de cálculo de distancias:", tiempo_distancias_activas)

######
from sklearn.metrics import silhouette_score

# Definir rangos de eps y min_samples para buscar
eps_values = np.arange(0.5, 1.5, 0.1)
min_samples_values = range(5, 20, 2)

mejor_eps = None
mejor_min_samples = None
mejor_silueta = -1

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric="precomputed")
        agrupacion = dbscan.fit(gower_dist)
        labels = agrupacion.labels_
        
        # Excluir el ruido (label -1) al calcular la silueta
        if len(set(labels)) > 1:
            silueta = silhouette_score(gower_dist, labels, metric="precomputed")
            if silueta > mejor_silueta:
                mejor_eps = eps
                mejor_min_samples = min_samples
                mejor_silueta = silueta

print(f"Mejor eps: {mejor_eps}")
print(f"Mejor min_samples: {mejor_min_samples}")
print(f"Mejor silueta: {mejor_silueta}")

######

# Ajuste del modelo DBSCAN
inicio = datetime.now()
dbscan = DBSCAN(eps=mejor_eps, min_samples=mejor_min_samples, metric="precomputed")
agrupacion = dbscan.fit(gower_dist)
fin = datetime.now()
tiempo_ajuste_modelo_activas = fin - inicio
print("Tiempo de ajuste del modelo:", tiempo_ajuste_modelo_activas)

# Asignar clusters al DataFrame
base_dbscan['coord'] = np.arange(1, len(base_dbscan) + 1)
base_dbscan['perfil'] = agrupacion.labels_

# Verifica cuántos clusters se formaron
num_clusters = len(set(base_dbscan['perfil']) - {-1})  # -1 es el ruido, no es un cluster real
print(f"Número de clusters formados: {num_clusters}")

# # Caracterización de las agrupaciones
# base_dbscan['perfil_descripcion'] = np.select(
#     [
#         base_dbscan['perfil'] == -1,
#         (base_dbscan['trabaja'] == "SI") & (base_dbscan['expediente_valido'] == "SI"),
#         (base_dbscan['trabaja'] == "NO") & (base_dbscan['expediente_valido'] == "SI"),
#         (base_dbscan['trabaja'] == "SI") & (base_dbscan['expediente_valido'] == "NO"),
#         (base_dbscan['trabaja'] == "NO") & (base_dbscan['expediente_valido'] == "NO"),
#     ],
#     [
#         "Ruido",
#         "T.si|E.si",
#         "T.no|E.si",
#         "T.si|E.no",
#         "T.no|E.no"
#     ],
#     default="Otro"
# )
# base_dbscan['perfil_descripcion'] = pd.Categorical(base_dbscan['perfil_descripcion'], 
#                                                    categories=["Ruido", "T.no|E.no", "T.si|E.no", "T.no|E.si", "T.si|E.si"], 
#                                                    ordered=True)

# # Crear particiones por clasificación
# base_dbscan_ruido = base_dbscan[base_dbscan['perfil_descripcion'] == "Ruido"]
# base_dbscan_ts_es = base_dbscan[base_dbscan['perfil_descripcion'] == "T.si|E.si"]
# base_dbscan_tn_es = base_dbscan[base_dbscan['perfil_descripcion'] == "T.no|E.si"]
# base_dbscan_ts_en = base_dbscan[base_dbscan['perfil_descripcion'] == "T.si|E.no"]
# base_dbscan_tn_en = base_dbscan[base_dbscan['perfil_descripcion'] == "T.no|E.no"]

# # Asignación de prioridades (requiere definición de la función 'prioridades')
# base_dbscan_ruido['prioridad'] = "Sin prioridad"
# base_dbscan_ts_es = prioridades(base_dbscan_ts_es)
# base_dbscan_tn_es = prioridades(base_dbscan_tn_es)
# base_dbscan_ts_en = prioridades(base_dbscan_ts_en)
# base_dbscan_tn_en = prioridades(base_dbscan_tn_en)

# base_dbscan_prioridades = pd.concat([base_dbscan_ruido, base_dbscan_ts_es, base_dbscan_tn_es, 
#                                      base_dbscan_ts_en, base_dbscan_tn_en])

# # Asignación de prioridades conjuntas (requiere definición de la función 'prioridades_conjuntas')
# base_dbscan_prioridades = prioridades_conjuntas(base_dbscan_prioridades)

# # Calidad de los agrupamientos (cálculo de siluetas)
# inicio = datetime.now()

# # Verificación de las columnas antes de calcular las siluetas
# print("Columnas del DataFrame antes de calcular las siluetas:", base_dbscan_prioridades.columns)

# if 'perfil' in base_dbscan_prioridades.columns:
#     # Verifica el rango de columnas que estás pasando a la función
#     print("Número de columnas en el DataFrame:", base_dbscan_prioridades.shape[1])
#     columnas_esperadas = base_dbscan_prioridades.iloc[:, 1:7].columns
#     print("Columnas esperadas para el cálculo de siluetas:", columnas_esperadas)
    
#     # Verifica si el índice del DataFrame es adecuado
#     print("Índices del DataFrame:", base_dbscan_prioridades.index)

#     # Llama a la función silueta_promedio
#     def silueta_promedio(X, labels):
#         """Calcula la silueta promedio para cada etiqueta"""
#         try:
#             sil_samples = silhouette_samples(X, labels, metric='precomputed')
#             return pd.DataFrame({'siluetas_modelo': sil_samples}, index=labels.index)
#         except Exception as e:
#             print("Error en el cálculo de la silueta:", e)
#             return None

#     siluetas_modelo = silueta_promedio(base_dbscan_prioridades.iloc[:, 1:7], base_dbscan_prioridades['perfil'])
# else:
#     print("La columna 'perfil' no está en el DataFrame")
#     siluetas_modelo = None

# fin = datetime.now()
# tiempo_siluetas_originales = fin - inicio
# print("Tiempo de cálculo de siluetas:", tiempo_siluetas_originales)

# # Asignar siluetas al DataFrame (si no hubo problemas en el cálculo)
# if siluetas_modelo is not None:
#     base_dbscan_prioridades = pd.concat([base_dbscan_prioridades, siluetas_modelo], axis=1)

# # Calcular la silueta promedio por perfil, solo si las siluetas fueron calculadas
# if 'siluetas_modelo' in base_dbscan_prioridades.columns:
#     silueta_modelo = base_dbscan_prioridades.groupby('perfil').agg(silueta_por_perfil=('siluetas_modelo', 'mean'))
#     print("Silueta promedio por perfil:")
#     print(silueta_modelo)

#     # Silueta promedio de los perfiles, excluyendo el ruido (perfil -1)
#     mean_silueta = silueta_modelo.loc[silueta_modelo.index != -1, 'silueta_por_perfil'].mean()
#     print("Silueta promedio excluyendo el ruido:", mean_silueta)
# else:
#     print("No se calcularon las siluetas, por lo tanto no se puede calcular la silueta promedio por perfil.")


Tiempo de cálculo de distancias: 0:01:58.137038
Mejor eps: None
Mejor min_samples: None
Mejor silueta: -1


InvalidParameterError: The 'eps' parameter of DBSCAN must be a float in the range (0.0, inf). Got None instead.

In [42]:
tabla_absoluta = pd.crosstab(index=base_dbscan['perfil_descripcion'], columns='Frecuencia')
tabla_relativa = pd.crosstab(index=base_dbscan['perfil_descripcion'], columns='Proporción', normalize='columns')
tabla_combinada = pd.concat([tabla_absoluta, tabla_relativa], axis=1)
tabla_combinada.columns = ['Frecuencia', 'Proporción']
print(tabla_combinada)

                    Frecuencia  Proporción
perfil_descripcion                        
Ruido                     4040    0.203896
T.no|E.no                 1074    0.054204
T.no|E.si                 2975    0.150146
T.si|E.no                 2463    0.124306
T.si|E.si                 9262    0.467447


In [ ]:
base_dbscan

In [ ]:
base_dbscan_prioridades